In [1]:
%env LD_LIBRARY_PATH=/usr/tce/packages/texlive/texlive-20220321/lib:/usr/WS2/haridev/dftracer/venv/lib/python3.9/site-packages/dftracer/lib:/usr/WS2/haridev/dftracer/venv/lib/python3.9/site-packages/dftracer/lib64:/usr/WS2/haridev/dftracer/venv/lib/python3.9/site-packages/dftracer/lib:/usr/WS2/haridev/dftracer/venv/lib/python3.9/site-packages/dftracer/lib64:/usr/WS2/haridev/dftracer/venv/lib

env: LD_LIBRARY_PATH=/usr/tce/packages/texlive/texlive-20220321/lib:/usr/WS2/haridev/dftracer/venv/lib/python3.9/site-packages/dftracer/lib:/usr/WS2/haridev/dftracer/venv/lib/python3.9/site-packages/dftracer/lib64:/usr/WS2/haridev/dftracer/venv/lib/python3.9/site-packages/dftracer/lib:/usr/WS2/haridev/dftracer/venv/lib/python3.9/site-packages/dftracer/lib64:/usr/WS2/haridev/dftracer/venv/lib


In [2]:
import recorder_viz
from recorder_viz import RecorderReader
#import darshan
from glob import glob
import pandas as pd
print(f"pd {pd.__version__}")
import dask
import dask.dataframe as dd
print(f"dask {dask.__version__}")
import pyarrow as pa
print(f"pa {pa.__version__}")
import numpy as np
print(f"np {np.__version__}")
from itertools import chain

from dask.distributed import Client, LocalCluster, progress, wait
from dask.distributed import Future, get_client
from typing import Tuple, Union
import os

pd 2.1.2


/var/tmp/haridev/ipykernel_2335677/3516059025.py:8: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be reported on the Dask issue tracker
https://github.com/dask/dask/issues 

  import dask.dataframe as dd


dask 2024.2.0
pa 15.0.0
np 1.26.1


In [3]:
folder="/usr/workspace/haridev/dftracer/logs/recorder"
app="dlio_40_10"
file=""

In [4]:
logfile=f"{folder}/{app}/{file}"

In [5]:
!ls $logfile

0.cfg  0.cst  0.ts  recorder.mt  VERSION


In [6]:
def get_json(func, ts, dur, rank):
    d = {}
    #print(location, start)
    d["name"] = func
    d["cat"] = "Recorder"
    d["ts"] = int(ts)
    d["dur"] = int(dur)
    d["pid"] = rank
    d["tid"] = 0
    #print(d)
    return d
import time

start = time.time()
def read_trace(trace_name):
    map_events = {}
    count = 0
    reader = RecorderReader(trace_name)
    func_list = reader.funcs
    for rank, records in enumerate(reader.records):
        lm = reader.LMs[rank]
        for record in records:
            if len(func_list) > record.func_id:
                func_name = func_list[record.func_id]
                if record.func_id > 0 and "MPI" not in func_name:
                    yield get_json(func_name, 0, 10, rank)
            

In [7]:
initialized = False

In [8]:
if not initialized:
    workers = 1
    cluster = LocalCluster(n_workers=workers, processes=True, threads_per_worker=1)  # Launches a scheduler and workers locally
    client = Client(cluster)  # Connect to distributed cluster and override default
    print(client)
    initialized = True

/usr/WS2/haridev/dftracer/venv/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44985 instead
  warnings.warn(


<Client: 'tcp://127.0.0.1:38597' processes=1 threads=1, memory=251.40 GiB>


2024-02-13 21:34:31,444 - distributed.nanny - WARNING - Restarting worker
2024-02-13 21:34:33,594 - distributed.nanny - WARNING - Restarting worker
2024-02-13 21:34:35,702 - distributed.nanny - WARNING - Restarting worker
2024-02-13 21:34:37,790 - distributed.nanny - WARNING - Restarting worker


In [9]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:44985/status,
Dashboard: http://127.0.0.1:44985/status,Workers: 1
Total threads: 1,Total memory: 251.40 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38597,Workers: 1
Dashboard: http://127.0.0.1:44985/status,Total threads: 1
Started: Just now,Total memory: 251.40 GiB
Comm: tcp://127.0.0.1:43613,Total threads: 1
Dashboard: http://127.0.0.1:38241/status,Memory: 251.40 GiB
Nanny: tcp://127.0.0.1:32877,


In [10]:
from dask.diagnostics import ProgressBar
from dask.distributed import progress
ProgressBar().register()

In [11]:
start = time.time()
file_pattern = glob(logfile)
create_bag = dask.bag.from_delayed([dask.delayed(read_trace)(file) 
                                                for file in file_pattern])
columns = {'name':"string", 'cat': "string",
           'pid': "string",'tid': "string",
           'dur': "uint64", 'ts': "uint64"}
events = create_bag.to_dataframe(meta=columns)
#events.head()
n_partition = 1
events = events.repartition(npartitions=n_partition).persist()
progress(events)
_ = wait(events)

22.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [12]:
#events.query("cat == 'RegionRole.FILE_IO'").groupby("name").count().compute()

In [13]:
#len(events)

In [14]:
events.tail()

KilledWorker: Attempted to run task ('bag-from-delayed-read_trace-to_dataframe-06e46d568c63791fd6b4cfe2ec1b9b1d', 0) on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:44645. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.